## Part 3

## Main Import Commands

In [ ]:
from utils.sbert import* # Importing the 'mean_pooling' function and the 'SentenceBertClass' class, serving a crucial part of the model

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy
from tqdm.notebook import tqdm
import plotly.express as px
import os 
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
 
from transformers import AutoTokenizer, AutoModel 
nlp = spacy.load('da_core_news_lg')

In [ ]:
# Modify the following path or name to the proper location and name of the trained model file 
model_path = "models/L03_1860.pth"

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

from transformers import AutoTokenizer, AutoModel

model_name = "sentence-transformers/paraphrase-MiniLM-L3-v2"

extractive_model = SentenceBertClass() 
extractive_model.load_state_dict(torch.load(model_path, map_location=torch.device(device)))
extractive_model.eval(); 

In [ ]:
# Tokenize text as required by BERT based models

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-MiniLM-L3-v2')

def get_tokens(text, tokenizer):
  inputs = tokenizer.batch_encode_plus(
            text, 
            add_special_tokens=True,
            max_length=512,
            padding="max_length",
            return_token_type_ids=True,
            truncation=True
        )
  ids = inputs['input_ids']
  mask = inputs['attention_mask']
  return ids, mask

In [ ]:
# Get predictions given an array of sentences and their corresponding documents
def predict(model,sents, doc):
  sent_id, sent_mask = get_tokens(sents,tokenizer)
  sent_id, sent_mask = torch.tensor(sent_id, dtype=torch.long),torch.tensor(sent_mask, dtype=torch.long)
 
  doc_id, doc_mask = get_tokens([doc],tokenizer)
  doc_id, doc_mask = doc_id * len(sents), doc_mask* len(sents)
  doc_id, doc_mask = torch.tensor(doc_id, dtype=torch.long),torch.tensor(doc_mask, dtype=torch.long)

  preds = model(sent_id, doc_id, sent_mask, doc_mask)
  return preds

In [ ]:
def summarize(doc, model, min_sentence_length=18, top_k=3, batch_size=3):
  doc = doc.replace("\n","")
  doc_sentences = []
  for sent in nlp(doc).sents:
    if len(sent) > min_sentence_length: 
      doc_sentences.append(str(sent))
  
  doc_id, doc_mask = get_tokens([doc],tokenizer)
  doc_id, doc_mask = doc_id * batch_size, doc_mask* batch_size
  doc_id, doc_mask = torch.tensor(doc_id, dtype=torch.long),torch.tensor(doc_mask, dtype=torch.long)

  scores = [] 
  # Run predictions using some batch size
  for i in tqdm(range(int(len(doc_sentences) / batch_size) + 1)):
    batch_start = i*batch_size  
    batch_end = (i+1) * batch_size if (i+1) * batch_size < len(doc) else len(doc)-1
    batch = doc_sentences[batch_start: batch_end]
    if batch:
      preds = predict(model, batch, doc) 
      scores = scores + preds.tolist() 
 
  sent_pred_list = [{"sentence": doc_sentences[i], "score": scores[i][0], "index":i} for i in range(len(doc_sentences))]
  sorted_sentences = sorted(sent_pred_list, key=lambda k: k['score'], reverse=True) 

  sorted_result = sorted_sentences[:top_k] 
  sorted_result = sorted(sorted_result, key=lambda k: k['index']) 
  
  summary = [ x["sentence"] for x in sorted_result]
  summary = " ".join(summary)

  return summary, scores, doc_sentences


In [ ]:
# Sampling an article for summary generation

arts = pd.read_csv('articles_summaries.csv')
article = arts['article'][0]
article

In [ ]:
summary, scores, sentences = summarize(article, extractive_model, min_sentence_length=18, top_k=3, batch_size=4)
summary

In [ ]:
# Visualising sentence scores

sent_length = 18
score_vals = ([x[0] for x in scores] ) 
sub_sents = [ str(round(score_vals[i],2)) + " : " + sentences[i][:sent_length] + " ..  " for i in range(len(sentences))] 
with plt.xkcd():
    plt.figure(figsize=(16,10));
    plt.barh(sub_sents[::-1], score_vals[::-1]);

## Model Evaluation

- Create a dataframe to store the ROUGE-1, ROUGE-2 and ROUGE-L precision, recall and F1 scores
- Averaging the dataframe containing the summation of scores

In [ ]:
data = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
 
gen2labdf = pd.DataFrame(data, columns = ['rouge-1', 'rouge-2', 'rouge-l'], index = ["r", "p", "f"])
tot_gen2labdf = pd.DataFrame(data, columns = ['rouge-1', 'rouge-2', 'rouge-l'], index = ["r", "p", "f"])

In [ ]:
sum_dir = ""

# Loading the saved test-set
test_df = pd.read_json(sum_dir +"data/test_bdf_da1855.json").reset_index()

In [ ]:
from rouge import Rouge
r = Rouge()

recall1_list = []
precision1_list = []
f11_list = []
recall2_list = []
precision2_list = []
f12_list = []
recall3_list = []
precision3_list = []
f13_list = []

for i in range(len(test_df)):
    art = test_df['docs'][i]
    example_summ = test_df['sents'][i]
    summ, _, _ = summarize(art, extractive_model, min_sentence_length=18, top_k=3, batch_size=4) # min_sentence_length kept constant at 18 to align with the test-set
    genlabi = r.get_scores(summ, example_summ)
    dfgli = pd.DataFrame(genlabi[0])
    gen2labdf=dfgli
    tot_gen2labdf+=dfgli
    gen2labdf = gen2labdf
    recall1_list.append(gen2labdf.iloc[0][0])
    recall2_list.append(gen2labdf.iloc[0][1])
    recall3_list.append(gen2labdf.iloc[0][2])
    precision1_list.append(gen2labdf.iloc[1][0])
    precision2_list.append(gen2labdf.iloc[1][1])
    precision3_list.append(gen2labdf.iloc[1][2])
    f11_list.append(gen2labdf.iloc[2][0])
    f12_list.append(gen2labdf.iloc[2][1])
    f13_list.append(gen2labdf.iloc[2][2])

In [ ]:
# Final result of the experiment

tot_gen2labdf/len(test_df)

## Plotting Results

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,5))
ax1.boxplot(recall1_list)
ax2.boxplot(precision1_list)
ax1.title.set_text("Recall r1")
ax2.title.set_text("Precision r1")
fig.tight_layout()
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,5))
ax1.boxplot(recall2_list)
ax2.boxplot(precision2_list)
ax1.title.set_text("Recall r2")
ax2.title.set_text("Precision r2")
fig.tight_layout()
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,5))
ax1.boxplot(recall3_list)
ax2.boxplot(precision3_list)
ax1.title.set_text("Recall r-L")
ax2.title.set_text("Precision r-L")
fig.tight_layout()
plt.show()

In [ ]:
plt.boxplot ([f11_list, f12_list, f13_list])
plt.title("F1 Score Distribution")
plt.xticks([1, 2, 3], ['ROUGE-1', 'ROUGE-2', 'ROUGE-L'])
plt.show()